# Stored Procedures and Triggers in Snowflake

## Learning Objectives
- Understand what stored procedures are and why they're used
- Learn the basics of creating and executing stored procedures
- Master parameters and variables in stored procedures
- Implement control flow with IF-ELSE statements
- Handle errors using TRY-CATCH blocks
- Follow best practices for styling stored procedures
- Understand what triggers are and their use cases
- Learn to create and manage triggers in Snowflake


## 1. Introduction to Stored Procedures

A **stored procedure** is a pre-compiled collection of SQL statements and procedural logic that is stored in the database. It can be executed multiple times with different parameters, making it reusable and efficient.

### Key Characteristics:
- **Pre-compiled**: Stored in the database and compiled once
- **Reusable**: Can be called multiple times with different parameters
- **Encapsulation**: Groups related SQL statements together
- **Performance**: Reduces network traffic and improves execution speed
- **Security**: Can control access to data through procedures

### Why Use Stored Procedures?
1. **Code Reusability**: Write once, use many times
2. **Performance**: Pre-compiled and optimized
3. **Maintainability**: Centralized business logic
4. **Security**: Control data access through procedures
5. **Complex Logic**: Handle complex business rules in the database
6. **Transaction Management**: Group multiple operations into transactions


## 2. Basics of Stored Procedures

In Snowflake, stored procedures are written using JavaScript for the procedural logic, but we'll focus on SQL-based procedures using Snowflake's SQL scripting capabilities. Snowflake supports stored procedures using JavaScript, but we can also use SQL scripting for simpler cases.

### Basic Syntax:
```sql
CREATE OR REPLACE PROCEDURE procedure_name()
RETURNS STRING
LANGUAGE SQL
AS
$$
    -- SQL statements here
    SELECT 'Hello World';
$$;
```

### Key Components:
- **CREATE OR REPLACE PROCEDURE**: Creates or replaces the procedure
- **procedure_name()**: Name of the procedure
- **RETURNS**: Data type of the return value
- **LANGUAGE SQL**: Specifies the language (SQL or JavaScript)
- **AS $$ ... $$**: Delimiter for the procedure body


In [ ]:
-- First, let's create sample tables for our examples
-- Create a customers table
CREATE OR REPLACE TABLE customers (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    city VARCHAR(50),
    country VARCHAR(50),
    registration_date DATE,
    customer_tier VARCHAR(20) DEFAULT 'Bronze'
);

-- Create an orders table
CREATE OR REPLACE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL(10, 2),
    status VARCHAR(20),
    product_category VARCHAR(50)
);

-- Create a products table
CREATE OR REPLACE TABLE products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(100),
    category VARCHAR(50),
    price DECIMAL(10, 2),
    stock_quantity INT
);

-- Insert dummy data into customers table
INSERT INTO customers VALUES
(1, 'John', 'Doe', 'john.doe@email.com', 'New York', 'USA', '2023-01-15', 'Gold'),
(2, 'Jane', 'Smith', 'jane.smith@email.com', 'London', 'UK', '2023-02-20', 'Silver'),
(3, 'Bob', 'Johnson', 'bob.johnson@email.com', 'Toronto', 'Canada', '2023-03-10', 'Bronze'),
(4, 'Alice', 'Williams', 'alice.williams@email.com', 'Sydney', 'Australia', '2023-04-05', 'Gold'),
(5, 'Charlie', 'Brown', 'charlie.brown@email.com', 'New York', 'USA', '2023-05-12', 'Silver');

-- Insert dummy data into orders table
INSERT INTO orders VALUES
(101, 1, '2023-06-01', 150.00, 'Completed', 'Electronics'),
(102, 1, '2023-07-15', 250.50, 'Completed', 'Clothing'),
(103, 2, '2023-06-20', 75.25, 'Pending', 'Books'),
(104, 3, '2023-08-01', 320.00, 'Completed', 'Electronics'),
(105, 4, '2023-08-10', 180.75, 'Completed', 'Clothing'),
(106, 1, '2023-09-05', 95.50, 'Pending', 'Books'),
(107, 5, '2023-09-12', 210.00, 'Completed', 'Electronics');

-- Insert dummy data into products table
INSERT INTO products VALUES
(1001, 'Laptop', 'Electronics', 999.99, 50),
(1002, 'T-Shirt', 'Clothing', 29.99, 200),
(1003, 'Novel', 'Books', 15.99, 150),
(1004, 'Smartphone', 'Electronics', 699.99, 75),
(1005, 'Jeans', 'Clothing', 49.99, 100);


### Example 1: Simple Stored Procedure

Let's create a simple stored procedure that returns a greeting message.


In [ ]:
-- Example 1: Simple stored procedure that returns a message
CREATE OR REPLACE PROCEDURE greet_user()
RETURNS STRING
LANGUAGE SQL
AS
$$
    SELECT 'Hello! Welcome to Snowflake Stored Procedures';
$$;

-- Execute the stored procedure
CALL greet_user();


### Example 2: Stored Procedure with SQL Operations

A stored procedure that performs a simple query operation.


In [ ]:
-- Example 2: Stored procedure that returns customer count
CREATE OR REPLACE PROCEDURE get_customer_count()
RETURNS INTEGER
LANGUAGE SQL
AS
$$
    SELECT COUNT(*) FROM customers;
$$;

-- Execute the stored procedure
CALL get_customer_count();


## 3. Parameters in Stored Procedures

Parameters allow stored procedures to accept input values, making them flexible and reusable. Parameters can be of various data types (INT, VARCHAR, DATE, etc.).

### Parameter Syntax:
```sql
CREATE OR REPLACE PROCEDURE procedure_name(param1 TYPE, param2 TYPE)
RETURNS return_type
LANGUAGE SQL
AS
$$
    -- Use parameters in SQL statements
$$;
```

### Types of Parameters:
- **IN Parameters**: Input parameters (default in Snowflake)
- **OUT Parameters**: Output parameters (not directly supported in Snowflake SQL procedures, use RETURN instead)
- **INOUT Parameters**: Both input and output (not directly supported in Snowflake SQL procedures)


### Example 3: Stored Procedure with Single Parameter


In [ ]:
-- Example 3: Stored procedure with a parameter to get customer by ID
CREATE OR REPLACE PROCEDURE get_customer_by_id(customer_id_param INT)
RETURNS TABLE (
    customer_id INT,
    full_name VARCHAR,
    email VARCHAR,
    city VARCHAR,
    country VARCHAR
)
LANGUAGE SQL
AS
$$
    SELECT 
        customer_id,
        first_name || ' ' || last_name AS full_name,
        email,
        city,
        country
    FROM customers
    WHERE customer_id = customer_id_param;
$$;

-- Execute with parameter
CALL get_customer_by_id(1);


### Example 4: Stored Procedure with Multiple Parameters


In [ ]:
-- Example 4: Stored procedure with multiple parameters
-- Get orders within a date range for a specific customer
CREATE OR REPLACE PROCEDURE get_customer_orders(
    customer_id_param INT,
    start_date DATE,
    end_date DATE
)
RETURNS TABLE (
    order_id INT,
    order_date DATE,
    total_amount DECIMAL(10, 2),
    status VARCHAR(20)
)
LANGUAGE SQL
AS
$$
    SELECT 
        order_id,
        order_date,
        total_amount,
        status
    FROM orders
    WHERE customer_id = customer_id_param
        AND order_date BETWEEN start_date AND end_date
    ORDER BY order_date DESC;
$$;

-- Execute with multiple parameters
CALL get_customer_orders(1, '2023-01-01', '2023-12-31');


### Example 5: Stored Procedure with Default Parameters

Note: Snowflake SQL procedures don't support default parameters directly. We need to handle this using conditional logic or use JavaScript procedures. However, we can simulate default behavior using NULL checks.


In [ ]:
-- Example 5: Stored procedure with optional parameter (using NULL)
-- Get customers by country, or all customers if country is NULL
CREATE OR REPLACE PROCEDURE get_customers_by_country(country_param VARCHAR)
RETURNS TABLE (
    customer_id INT,
    full_name VARCHAR,
    email VARCHAR,
    city VARCHAR,
    country VARCHAR
)
LANGUAGE SQL
AS
$$
    SELECT 
        customer_id,
        first_name || ' ' || last_name AS full_name,
        email,
        city,
        country
    FROM customers
    WHERE country_param IS NULL OR country = country_param
    ORDER BY customer_id;
$$;

-- Execute with parameter
CALL get_customers_by_country('USA');

-- Execute without filtering (pass NULL)
CALL get_customers_by_country(NULL);


## 4. Variables in Stored Procedures

Variables allow you to store intermediate values within a stored procedure. In Snowflake SQL procedures, variables are declared using the `DECLARE` statement and assigned using `SET` or `:=`.

### Variable Syntax:
```sql
DECLARE
    variable_name VARIABLE_TYPE;
BEGIN
    SET variable_name = value;
    -- Use variable in SQL statements
END;
```

### Key Points:
- Variables must be declared before use
- Variables are local to the procedure
- Use `SET` or `:=` to assign values
- Variables can be used in SQL expressions


### Example 6: Using Variables in Stored Procedures


In [ ]:
-- Example 6: Stored procedure using variables
-- Calculate total order value for a customer and return formatted message
CREATE OR REPLACE PROCEDURE get_customer_order_summary(customer_id_param INT)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        total_orders INT;
        total_amount DECIMAL(10, 2);
        customer_name VARCHAR;
        result_message STRING;
    BEGIN
        -- Get customer name
        SELECT first_name || ' ' || last_name INTO customer_name
        FROM customers
        WHERE customer_id = customer_id_param;
        
        -- Get order count and total amount
        SELECT 
            COUNT(*),
            COALESCE(SUM(total_amount), 0)
        INTO total_orders, total_amount
        FROM orders
        WHERE customer_id = customer_id_param;
        
        -- Build result message
        SET result_message = customer_name || ' has placed ' || 
                            total_orders || ' orders with a total value of $' || 
                            total_amount;
        
        RETURN result_message;
    END;
$$;

-- Execute the procedure
CALL get_customer_order_summary(1);


### Example 7: Variables with Calculations


In [ ]:
-- Example 7: Using variables for calculations
-- Calculate discount based on customer tier
CREATE OR REPLACE PROCEDURE calculate_discount(
    customer_id_param INT,
    order_amount DECIMAL(10, 2)
)
RETURNS TABLE (
    customer_tier VARCHAR,
    original_amount DECIMAL(10, 2),
    discount_percent DECIMAL(5, 2),
    discount_amount DECIMAL(10, 2),
    final_amount DECIMAL(10, 2)
)
LANGUAGE SQL
AS
$$
    DECLARE
        tier VARCHAR(20);
        discount_pct DECIMAL(5, 2);
        discount_amt DECIMAL(10, 2);
        final_amt DECIMAL(10, 2);
    BEGIN
        -- Get customer tier
        SELECT customer_tier INTO tier
        FROM customers
        WHERE customer_id = customer_id_param;
        
        -- Calculate discount based on tier
        CASE tier
            WHEN 'Gold' THEN SET discount_pct = 15.00;
            WHEN 'Silver' THEN SET discount_pct = 10.00;
            WHEN 'Bronze' THEN SET discount_pct = 5.00;
            ELSE SET discount_pct = 0.00;
        END CASE;
        
        -- Calculate discount amount and final amount
        SET discount_amt = order_amount * (discount_pct / 100);
        SET final_amt = order_amount - discount_amt;
        
        -- Return results
        RETURN TABLE(
            SELECT 
                tier AS customer_tier,
                order_amount AS original_amount,
                discount_pct AS discount_percent,
                discount_amt AS discount_amount,
                final_amt AS final_amount
        );
    END;
$$;

-- Execute the procedure
CALL calculate_discount(1, 100.00);
CALL calculate_discount(2, 100.00);
CALL calculate_discount(3, 100.00);


## 5. Control Flow: IF-ELSE Statements

Control flow statements allow you to execute different code blocks based on conditions. IF-ELSE statements are essential for implementing business logic in stored procedures.

### IF-ELSE Syntax:
```sql
IF condition THEN
    -- statements
ELSEIF condition THEN
    -- statements
ELSE
    -- statements
END IF;
```

### Key Points:
- Conditions are evaluated in order
- First matching condition executes
- ELSE is optional
- Can nest IF statements


### Example 8: Simple IF-ELSE Statement


In [ ]:
-- Example 8: Simple IF-ELSE to check order status
CREATE OR REPLACE PROCEDURE check_order_status(order_id_param INT)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        order_status VARCHAR(20);
        status_message STRING;
    BEGIN
        -- Get order status
        SELECT status INTO order_status
        FROM orders
        WHERE order_id = order_id_param;
        
        -- Use IF-ELSE to determine message
        IF order_status = 'Completed' THEN
            SET status_message = 'Order ' || order_id_param || ' has been completed successfully.';
        ELSEIF order_status = 'Pending' THEN
            SET status_message = 'Order ' || order_id_param || ' is pending processing.';
        ELSE
            SET status_message = 'Order ' || order_id_param || ' has status: ' || order_status;
        END IF;
        
        RETURN status_message;
    END;
$$;

-- Execute the procedure
CALL check_order_status(101);
CALL check_order_status(103);


### Example 9: Complex IF-ELSE with Multiple Conditions


In [ ]:
-- Example 9: Update customer tier based on total order amount
CREATE OR REPLACE PROCEDURE update_customer_tier(customer_id_param INT)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        total_spent DECIMAL(10, 2);
        current_tier VARCHAR(20);
        new_tier VARCHAR(20);
        update_message STRING;
    BEGIN
        -- Get current tier
        SELECT customer_tier INTO current_tier
        FROM customers
        WHERE customer_id = customer_id_param;
        
        -- Calculate total amount spent
        SELECT COALESCE(SUM(total_amount), 0) INTO total_spent
        FROM orders
        WHERE customer_id = customer_id_param AND status = 'Completed';
        
        -- Determine new tier based on spending
        IF total_spent >= 500 THEN
            SET new_tier = 'Gold';
        ELSEIF total_spent >= 200 THEN
            SET new_tier = 'Silver';
        ELSE
            SET new_tier = 'Bronze';
        END IF;
        
        -- Update tier if it has changed
        IF current_tier != new_tier THEN
            UPDATE customers
            SET customer_tier = new_tier
            WHERE customer_id = customer_id_param;
            
            SET update_message = 'Customer tier updated from ' || current_tier || 
                                ' to ' || new_tier || 
                                ' (Total spent: $' || total_spent || ')';
        ELSE
            SET update_message = 'Customer tier remains ' || current_tier || 
                                ' (Total spent: $' || total_spent || ')';
        END IF;
        
        RETURN update_message;
    END;
$$;

-- Execute the procedure
CALL update_customer_tier(1);
CALL update_customer_tier(2);
CALL update_customer_tier(3);


### Example 10: Nested IF Statements


In [ ]:
-- Example 10: Nested IF statements for complex business logic
-- Process order with validation and tier-based discount
-- Note: discount_code can be NULL (passed explicitly, as Snowflake SQL procedures don't support default parameters)
CREATE OR REPLACE PROCEDURE process_order(
    order_id_param INT,
    discount_code VARCHAR
)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        order_exists INT;
        customer_id_val INT;
        order_amount DECIMAL(10, 2);
        customer_tier_val VARCHAR(20);
        final_discount DECIMAL(5, 2);
        result_message STRING;
    BEGIN
        -- Check if order exists
        SELECT COUNT(*) INTO order_exists
        FROM orders
        WHERE order_id = order_id_param;
        
        IF order_exists = 0 THEN
            RETURN 'Error: Order ' || order_id_param || ' does not exist.';
        ELSE
            -- Get order details
            SELECT customer_id, total_amount INTO customer_id_val, order_amount
            FROM orders
            WHERE order_id = order_id_param;
            
            -- Get customer tier
            SELECT customer_tier INTO customer_tier_val
            FROM customers
            WHERE customer_id = customer_id_val;
            
            -- Apply discount logic
            IF discount_code IS NOT NULL THEN
                IF discount_code = 'SPECIAL10' THEN
                    SET final_discount = 10.00;
                ELSEIF discount_code = 'VIP20' AND customer_tier_val = 'Gold' THEN
                    SET final_discount = 20.00;
                ELSE
                    SET final_discount = 0.00;
                END IF;
            ELSE
                -- Apply tier-based discount
                IF customer_tier_val = 'Gold' THEN
                    SET final_discount = 15.00;
                ELSEIF customer_tier_val = 'Silver' THEN
                    SET final_discount = 10.00;
                ELSE
                    SET final_discount = 5.00;
                END IF;
            END IF;
            
            SET result_message = 'Order ' || order_id_param || 
                                ' processed. Original: $' || order_amount ||
                                ', Discount: ' || final_discount || '%' ||
                                ', Final: $' || (order_amount * (1 - final_discount/100));
            
            RETURN result_message;
        END IF;
    END;
$$;

-- Execute the procedure
CALL process_order(101, NULL);
CALL process_order(102, 'SPECIAL10');
CALL process_order(104, 'VIP20');


### Example 11: Basic Error Handling


In [ ]:
-- Example 11: Basic error handling in stored procedure
-- Insert a new order with validation
CREATE OR REPLACE PROCEDURE insert_order(
    order_id_param INT,
    customer_id_param INT,
    order_date_param DATE,
    total_amount_param DECIMAL(10, 2),
    status_param VARCHAR(20)
)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        customer_exists INT;
        result_message STRING;
    BEGIN
        -- Check if customer exists
        SELECT COUNT(*) INTO customer_exists
        FROM customers
        WHERE customer_id = customer_id_param;
        
        IF customer_exists = 0 THEN
            RETURN 'Error: Customer ' || customer_id_param || ' does not exist.';
        END IF;
        
        -- Try to insert order
        BEGIN
            INSERT INTO orders (order_id, customer_id, order_date, total_amount, status)
            VALUES (order_id_param, customer_id_param, order_date_param, total_amount_param, status_param);
            
            SET result_message = 'Order ' || order_id_param || ' inserted successfully.';
            RETURN result_message;
        EXCEPTION
            WHEN OTHER THEN
                RETURN 'Error inserting order: ' || SQLERRM;
        END;
    END;
$$;

-- Execute with valid data
CALL insert_order(108, 1, '2023-10-01', 150.00, 'Pending');

-- Execute with invalid customer (will return error message)
CALL insert_order(109, 999, '2023-10-01', 150.00, 'Pending');

-- Execute with duplicate order_id (will catch exception)
CALL insert_order(108, 1, '2023-10-02', 200.00, 'Pending');


### Example 12: Advanced Error Handling with Multiple Exception Types


In [ ]:
-- Example 12: Advanced error handling
-- Update product stock with comprehensive error handling
CREATE OR REPLACE PROCEDURE update_product_stock(
    product_id_param INT,
    quantity_change INT
)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        product_exists INT;
        current_stock INT;
        new_stock INT;
        result_message STRING;
    BEGIN
        -- Validate input
        IF quantity_change = 0 THEN
            RETURN 'Error: Quantity change cannot be zero.';
        END IF;
        
        -- Check if product exists
        SELECT COUNT(*) INTO product_exists
        FROM products
        WHERE product_id = product_id_param;
        
        IF product_exists = 0 THEN
            RETURN 'Error: Product ' || product_id_param || ' does not exist.';
        END IF;
        
        -- Get current stock
        SELECT stock_quantity INTO current_stock
        FROM products
        WHERE product_id = product_id_param;
        
        -- Calculate new stock
        SET new_stock = current_stock + quantity_change;
        
        -- Validate stock cannot go negative
        IF new_stock < 0 THEN
            RETURN 'Error: Insufficient stock. Current: ' || current_stock || 
                   ', Requested change: ' || quantity_change;
        END IF;
        
        -- Try to update stock
        BEGIN
            UPDATE products
            SET stock_quantity = new_stock
            WHERE product_id = product_id_param;
            
            SET result_message = 'Product ' || product_id_param || 
                               ' stock updated. Old: ' || current_stock ||
                               ', New: ' || new_stock;
            RETURN result_message;
        EXCEPTION
            WHEN OTHER THEN
                RETURN 'Error updating product stock: ' || SQLERRM;
        END;
    END;
$$;

-- Execute with valid data
CALL update_product_stock(1001, -10);  -- Reduce stock
CALL update_product_stock(1001, 5);    -- Increase stock

-- Execute with invalid product
CALL update_product_stock(9999, 10);

-- Execute with insufficient stock
CALL update_product_stock(1001, -1000);


### Example 13: Transaction Management with Error Handling


In [ ]:
-- Example 13: Transaction management with error handling
-- Create order and update customer tier in a transaction
CREATE OR REPLACE PROCEDURE create_order_with_tier_update(
    order_id_param INT,
    customer_id_param INT,
    order_date_param DATE,
    total_amount_param DECIMAL(10, 2),
    status_param VARCHAR(20)
)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        customer_exists INT;
        total_spent DECIMAL(10, 2);
        new_tier VARCHAR(20);
        result_message STRING;
    BEGIN
        -- Validate customer exists
        SELECT COUNT(*) INTO customer_exists
        FROM customers
        WHERE customer_id = customer_id_param;
        
        IF customer_exists = 0 THEN
            RETURN 'Error: Customer ' || customer_id_param || ' does not exist.';
        END IF;
        
        -- Begin transaction (implicit in Snowflake procedures)
        BEGIN
            -- Insert order
            INSERT INTO orders (order_id, customer_id, order_date, total_amount, status)
            VALUES (order_id_param, customer_id_param, order_date_param, total_amount_param, status_param);
            
            -- Calculate total spent (including new order if completed)
            SELECT COALESCE(SUM(total_amount), 0) INTO total_spent
            FROM orders
            WHERE customer_id = customer_id_param 
                AND status = 'Completed';
            
            -- Determine new tier
            IF total_spent >= 500 THEN
                SET new_tier = 'Gold';
            ELSEIF total_spent >= 200 THEN
                SET new_tier = 'Silver';
            ELSE
                SET new_tier = 'Bronze';
            END IF;
            
            -- Update customer tier
            UPDATE customers
            SET customer_tier = new_tier
            WHERE customer_id = customer_id_param;
            
            SET result_message = 'Order ' || order_id_param || 
                               ' created and customer tier updated to ' || new_tier;
            RETURN result_message;
            
        EXCEPTION
            WHEN OTHER THEN
                -- Transaction will be rolled back automatically
                RETURN 'Error: Transaction failed. ' || SQLERRM || 
                       ' All changes have been rolled back.';
        END;
    END;
$$;

-- Execute the procedure
CALL create_order_with_tier_update(110, 1, '2023-10-15', 300.00, 'Completed');


## 7. Styling Stored Procedures (Best Practices)

Writing well-styled stored procedures improves readability, maintainability, and reduces errors. Follow these best practices:

### 1. Naming Conventions
- Use descriptive, meaningful names
- Use consistent prefixes (e.g., `sp_`, `proc_`, or no prefix)
- Use snake_case or camelCase consistently
- Include action verbs (get_, insert_, update_, delete_)

### 2. Code Organization
- Group related statements together
- Use comments to explain complex logic
- Keep procedures focused on a single task
- Break complex procedures into smaller ones

### 3. Formatting
- Consistent indentation (2 or 4 spaces)
- Align similar statements
- Use blank lines to separate logical sections
- Format SQL statements clearly

### 4. Documentation
- Add header comments explaining purpose
- Document parameters
- Document return values
- Include usage examples

### 5. Error Handling
- Always include error handling
- Provide meaningful error messages
- Log errors appropriately
- Handle edge cases

### 6. Performance
- Use appropriate data types
- Avoid unnecessary operations
- Use indexes effectively
- Minimize data transfers


### Example 14: Well-Styled Stored Procedure


In [ ]:
-- Example 14: Well-styled stored procedure following best practices
/*
    Procedure: get_customer_order_history
    
    Purpose: Retrieves complete order history for a customer including
             order details, totals, and customer information.
    
    Parameters:
        - customer_id_param: INT - The ID of the customer
    
    Returns: TABLE with order history details
    
    Usage:
        CALL get_customer_order_history(1);
    
    Author: Data Engineering Team
    Created: 2023-10-01
    Modified: 2023-10-01
*/
CREATE OR REPLACE PROCEDURE get_customer_order_history(customer_id_param INT)
RETURNS TABLE (
    order_id INT,
    order_date DATE,
    total_amount DECIMAL(10, 2),
    status VARCHAR(20),
    customer_name VARCHAR,
    customer_tier VARCHAR(20),
    total_orders INT,
    lifetime_value DECIMAL(10, 2)
)
LANGUAGE SQL
AS
$$
    DECLARE
        -- Variable declarations
        customer_name_val VARCHAR;
        customer_tier_val VARCHAR(20);
        total_orders_val INT;
        lifetime_value_val DECIMAL(10, 2);
    BEGIN
        -- Validate input parameter
        IF customer_id_param IS NULL OR customer_id_param <= 0 THEN
            RETURN TABLE(
                SELECT 
                    NULL::INT AS order_id,
                    NULL::DATE AS order_date,
                    NULL::DECIMAL(10, 2) AS total_amount,
                    'INVALID_CUSTOMER_ID'::VARCHAR AS status,
                    NULL::VARCHAR AS customer_name,
                    NULL::VARCHAR(20) AS customer_tier,
                    NULL::INT AS total_orders,
                    NULL::DECIMAL(10, 2) AS lifetime_value
                WHERE FALSE
            );
        END IF;
        
        -- Get customer information
        SELECT 
            first_name || ' ' || last_name,
            customer_tier
        INTO customer_name_val, customer_tier_val
        FROM customers
        WHERE customer_id = customer_id_param;
        
        -- Check if customer exists
        IF customer_name_val IS NULL THEN
            RETURN TABLE(
                SELECT 
                    NULL::INT AS order_id,
                    NULL::DATE AS order_date,
                    NULL::DECIMAL(10, 2) AS total_amount,
                    'CUSTOMER_NOT_FOUND'::VARCHAR AS status,
                    NULL::VARCHAR AS customer_name,
                    NULL::VARCHAR(20) AS customer_tier,
                    NULL::INT AS total_orders,
                    NULL::DECIMAL(10, 2) AS lifetime_value
                WHERE FALSE
            );
        END IF;
        
        -- Calculate customer statistics
        SELECT 
            COUNT(*),
            COALESCE(SUM(total_amount), 0)
        INTO total_orders_val, lifetime_value_val
        FROM orders
        WHERE customer_id = customer_id_param;
        
        -- Return order history with customer details
        RETURN TABLE(
            SELECT 
                o.order_id,
                o.order_date,
                o.total_amount,
                o.status,
                customer_name_val AS customer_name,
                customer_tier_val AS customer_tier,
                total_orders_val AS total_orders,
                lifetime_value_val AS lifetime_value
            FROM orders o
            WHERE o.customer_id = customer_id_param
            ORDER BY o.order_date DESC
        );
        
    EXCEPTION
        WHEN OTHER THEN
            -- Return error information
            RETURN TABLE(
                SELECT 
                    NULL::INT AS order_id,
                    NULL::DATE AS order_date,
                    NULL::DECIMAL(10, 2) AS total_amount,
                    ('ERROR: ' || SQLERRM)::VARCHAR AS status,
                    NULL::VARCHAR AS customer_name,
                    NULL::VARCHAR(20) AS customer_tier,
                    NULL::INT AS total_orders,
                    NULL::DECIMAL(10, 2) AS lifetime_value
                WHERE FALSE
            );
    END;
$$;

-- Execute the well-styled procedure
CALL get_customer_order_history(1);


### Trigger Concepts (General SQL)

In traditional SQL databases, triggers follow this syntax:

```sql
CREATE TRIGGER trigger_name
BEFORE|AFTER INSERT|UPDATE|DELETE
ON table_name
FOR EACH ROW|STATEMENT
BEGIN
    -- Trigger logic
END;
```

### Common Use Cases for Triggers:
1. **Audit Logging**: Track changes to sensitive data
2. **Data Validation**: Enforce business rules
3. **Automatic Calculations**: Update related tables
4. **Maintaining History**: Keep historical records
5. **Enforcing Constraints**: Complex validation rules
6. **Sending Notifications**: Alert on specific events


## 9. Trigger Use Cases and Snowflake Alternatives

Since Snowflake doesn't support triggers, we'll explore common trigger use cases and how to achieve similar functionality using Snowflake features.

### Use Case 1: Audit Logging

**Traditional Trigger Approach:**
- Trigger fires on INSERT/UPDATE/DELETE
- Logs changes to an audit table

**Snowflake Alternative:**
- Use Streams to capture changes
- Use Tasks to process changes
- Or use stored procedures with explicit logging


In [ ]:
-- Use Case 1: Audit Logging using Stored Procedure
-- Create audit log table
CREATE OR REPLACE TABLE order_audit_log (
    audit_id INT AUTOINCREMENT PRIMARY KEY,
    order_id INT,
    action_type VARCHAR(20),  -- INSERT, UPDATE, DELETE
    old_status VARCHAR(20),
    new_status VARCHAR(20),
    changed_by VARCHAR(100),
    change_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);

-- Stored procedure to log order changes
CREATE OR REPLACE PROCEDURE log_order_change(
    order_id_param INT,
    action_type_param VARCHAR(20),
    old_status_param VARCHAR(20),
    new_status_param VARCHAR(20),
    changed_by_param VARCHAR(100)
)
RETURNS STRING
LANGUAGE SQL
AS
$$
    BEGIN
        INSERT INTO order_audit_log (
            order_id,
            action_type,
            old_status,
            new_status,
            changed_by
        )
        VALUES (
            order_id_param,
            action_type_param,
            old_status_param,
            new_status_param,
            changed_by_param
        );
        
        RETURN 'Audit log entry created for order ' || order_id_param;
    EXCEPTION
        WHEN OTHER THEN
            RETURN 'Error logging change: ' || SQLERRM;
    END;
$$;

-- Example: Update order and log the change
CREATE OR REPLACE PROCEDURE update_order_with_audit(
    order_id_param INT,
    new_status_param VARCHAR(20),
    changed_by_param VARCHAR(100)
)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        old_status_val VARCHAR(20);
        result_message STRING;
    BEGIN
        -- Get current status
        SELECT status INTO old_status_val
        FROM orders
        WHERE order_id = order_id_param;
        
        IF old_status_val IS NULL THEN
            RETURN 'Error: Order ' || order_id_param || ' does not exist.';
        END IF;
        
        -- Update order
        UPDATE orders
        SET status = new_status_param
        WHERE order_id = order_id_param;
        
        -- Log the change
        CALL log_order_change(
            order_id_param,
            'UPDATE',
            old_status_val,
            new_status_param,
            changed_by_param
        );
        
        SET result_message = 'Order ' || order_id_param || 
                           ' updated from ' || old_status_val ||
                           ' to ' || new_status_param;
        RETURN result_message;
    END;
$$;

-- Execute the procedure
CALL update_order_with_audit(103, 'Completed', 'admin@company.com');

-- View audit log
SELECT * FROM order_audit_log ORDER BY change_timestamp DESC;


### Use Case 2: Automatic Calculations

**Traditional Trigger Approach:**
- Trigger fires on INSERT/UPDATE
- Automatically calculates and updates related fields

**Snowflake Alternative:**
- Use stored procedures that perform calculations
- Call procedures explicitly when needed
- Or use computed columns (if supported)


In [ ]:
-- Use Case 2: Automatic Calculations
-- Create a table to track customer statistics
CREATE OR REPLACE TABLE customer_statistics (
    customer_id INT PRIMARY KEY,
    total_orders INT DEFAULT 0,
    total_spent DECIMAL(10, 2) DEFAULT 0,
    average_order_value DECIMAL(10, 2) DEFAULT 0,
    last_order_date DATE,
    last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP()
);

-- Initialize statistics for existing customers
INSERT INTO customer_statistics (customer_id, total_orders, total_spent, average_order_value, last_order_date)
SELECT 
    c.customer_id,
    COUNT(o.order_id) AS total_orders,
    COALESCE(SUM(o.total_amount), 0) AS total_spent,
    COALESCE(AVG(o.total_amount), 0) AS average_order_value,
    MAX(o.order_date) AS last_order_date
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id;

-- Stored procedure to recalculate customer statistics
CREATE OR REPLACE PROCEDURE recalculate_customer_stats(customer_id_param INT)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        total_orders_val INT;
        total_spent_val DECIMAL(10, 2);
        avg_order_val DECIMAL(10, 2);
        last_order_date_val DATE;
        result_message STRING;
    BEGIN
        -- Calculate statistics
        SELECT 
            COUNT(*),
            COALESCE(SUM(total_amount), 0),
            COALESCE(AVG(total_amount), 0),
            MAX(order_date)
        INTO total_orders_val, total_spent_val, avg_order_val, last_order_date_val
        FROM orders
        WHERE customer_id = customer_id_param;
        
        -- Update or insert statistics
        MERGE INTO customer_statistics AS target
        USING (
            SELECT 
                customer_id_param AS customer_id,
                total_orders_val AS total_orders,
                total_spent_val AS total_spent,
                avg_order_val AS average_order_value,
                last_order_date_val AS last_order_date
        ) AS source
        ON target.customer_id = source.customer_id
        WHEN MATCHED THEN
            UPDATE SET
                total_orders = source.total_orders,
                total_spent = source.total_spent,
                average_order_value = source.average_order_value,
                last_order_date = source.last_order_date,
                last_updated = CURRENT_TIMESTAMP()
        WHEN NOT MATCHED THEN
            INSERT (customer_id, total_orders, total_spent, average_order_value, last_order_date)
            VALUES (source.customer_id, source.total_orders, source.total_spent, 
                   source.average_order_value, source.last_order_date);
        
        SET result_message = 'Statistics updated for customer ' || customer_id_param;
        RETURN result_message;
    EXCEPTION
        WHEN OTHER THEN
            RETURN 'Error updating statistics: ' || SQLERRM;
    END;
$$;

-- Enhanced order creation procedure that updates statistics
CREATE OR REPLACE PROCEDURE create_order_with_stats_update(
    order_id_param INT,
    customer_id_param INT,
    order_date_param DATE,
    total_amount_param DECIMAL(10, 2),
    status_param VARCHAR(20)
)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        result_message STRING;
    BEGIN
        -- Insert order
        INSERT INTO orders (order_id, customer_id, order_date, total_amount, status)
        VALUES (order_id_param, customer_id_param, order_date_param, total_amount_param, status_param);
        
        -- Update customer statistics (like a trigger would)
        CALL recalculate_customer_stats(customer_id_param);
        
        SET result_message = 'Order ' || order_id_param || ' created and statistics updated.';
        RETURN result_message;
    EXCEPTION
        WHEN OTHER THEN
            RETURN 'Error creating order: ' || SQLERRM;
    END;
$$;

-- Execute the procedure
CALL create_order_with_stats_update(111, 1, '2023-10-20', 175.50, 'Completed');

-- View updated statistics
SELECT * FROM customer_statistics WHERE customer_id = 1;


### Use Case 3: Data Validation

**Traditional Trigger Approach:**
- BEFORE trigger validates data
- Prevents invalid data from being inserted/updated

**Snowflake Alternative:**
- Use stored procedures with validation logic
- Call procedures instead of direct INSERT/UPDATE
- Or use CHECK constraints for simple validations


In [ ]:
-- Use Case 3: Data Validation
-- Stored procedure with comprehensive validation (like a BEFORE trigger)
CREATE OR REPLACE PROCEDURE insert_validated_order(
    order_id_param INT,
    customer_id_param INT,
    order_date_param DATE,
    total_amount_param DECIMAL(10, 2),
    status_param VARCHAR(20)
)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        customer_exists INT;
        validation_errors STRING DEFAULT '';
    BEGIN
        -- Validate order_id is positive
        IF order_id_param IS NULL OR order_id_param <= 0 THEN
            SET validation_errors = validation_errors || 'Order ID must be positive. ';
        END IF;
        
        -- Validate customer exists
        SELECT COUNT(*) INTO customer_exists
        FROM customers
        WHERE customer_id = customer_id_param;
        
        IF customer_exists = 0 THEN
            SET validation_errors = validation_errors || 'Customer does not exist. ';
        END IF;
        
        -- Validate order_date is not in the future
        IF order_date_param > CURRENT_DATE() THEN
            SET validation_errors = validation_errors || 'Order date cannot be in the future. ';
        END IF;
        
        -- Validate total_amount is positive
        IF total_amount_param IS NULL OR total_amount_param <= 0 THEN
            SET validation_errors = validation_errors || 'Total amount must be positive. ';
        END IF;
        
        -- Validate status
        IF status_param NOT IN ('Pending', 'Completed', 'Cancelled', 'Shipped') THEN
            SET validation_errors = validation_errors || 'Invalid status. Must be Pending, Completed, Cancelled, or Shipped. ';
        END IF;
        
        -- If validation errors exist, return them
        IF validation_errors != '' THEN
            RETURN 'Validation Error: ' || validation_errors;
        END IF;
        
        -- All validations passed, insert the order
        BEGIN
            INSERT INTO orders (order_id, customer_id, order_date, total_amount, status)
            VALUES (order_id_param, customer_id_param, order_date_param, total_amount_param, status_param);
            
            RETURN 'Order ' || order_id_param || ' inserted successfully.';
        EXCEPTION
            WHEN OTHER THEN
                RETURN 'Error inserting order: ' || SQLERRM;
        END;
    END;
$$;

-- Execute with valid data
CALL insert_validated_order(112, 2, '2023-10-15', 125.00, 'Pending');

-- Execute with invalid data (will return validation errors)
CALL insert_validated_order(113, 999, '2024-12-31', -50.00, 'InvalidStatus');


## Summary

### Key Takeaways:

1. **Stored Procedures**:
   - Pre-compiled, reusable SQL code blocks
   - Improve performance and maintainability
   - Encapsulate business logic

2. **Parameters**:
   - Make procedures flexible and reusable
   - Accept input values of various types
   - Enable dynamic behavior

3. **Variables**:
   - Store intermediate values
   - Enable complex calculations
   - Improve code readability

4. **Control Flow (IF-ELSE)**:
   - Implement conditional logic
   - Handle different scenarios
   - Support complex business rules

5. **Error Handling (TRY-CATCH)**:
   - Gracefully handle exceptions
   - Provide meaningful error messages
   - Ensure data integrity

6. **Best Practices**:
   - Follow naming conventions
   - Write clear documentation
   - Organize code logically
   - Always include error handling

7. **Triggers**:
   - Event-driven automatic execution
   - Not supported in Snowflake
   - Use stored procedures and explicit calls as alternatives


## Practice Problems

### Problem 1: Basic Stored Procedure
Create a stored procedure called `get_product_info` that accepts a product_id and returns the product name, category, price, and stock quantity.

**Expected Output Format:**
- Returns a table with columns: product_name, category, price, stock_quantity


### Problem 2: Stored Procedure with Variables
Create a stored procedure called `calculate_order_total` that:
- Accepts an order_id
- Calculates the total amount for that order
- If the total is greater than $200, applies a 10% discount
- Returns the original amount, discount amount (if any), and final amount

**Expected Output Format:**
- Returns a string with the calculation details


### Problem 3: Control Flow (IF-ELSE)
Create a stored procedure called `assign_customer_tier` that:
- Accepts a customer_id
- Calculates the total amount spent by the customer
- Assigns tier based on spending:
  - Gold: >= $500
  - Silver: >= $200 and < $500
  - Bronze: < $200
- Updates the customer's tier in the customers table
- Returns a message indicating the new tier

**Expected Output Format:**
- Returns a string message


### Problem 4: Error Handling
Create a stored procedure called `safe_delete_order` that:
- Accepts an order_id
- Checks if the order exists
- Checks if the order status is 'Pending' (only pending orders can be deleted)
- If valid, deletes the order
- If invalid, returns an appropriate error message
- Uses proper error handling

**Expected Output Format:**
- Returns a string with success or error message


### Problem 5: Complex Stored Procedure
Create a stored procedure called `process_customer_order` that:
- Accepts: customer_id, order_date, total_amount, status
- Validates that the customer exists
- Validates that total_amount is positive
- Validates that order_date is not in the future
- Generates a new order_id (use MAX(order_id) + 1)
- Inserts the new order
- Updates customer statistics (like a trigger would)
- Returns success message with order_id

**Expected Output Format:**
- Returns a string with order details


### Problem 6: Multiple Parameters and Complex Logic
Create a stored procedure called `get_customer_orders_summary` that:
- Accepts: customer_id, start_date, end_date
- Returns a summary table with:
  - Total number of orders in the date range
  - Total amount spent
  - Average order value
  - Number of completed orders
  - Number of pending orders
  - Customer name and tier

**Expected Output Format:**
- Returns a table with summary statistics


### Problem 7: Trigger-like Functionality
Create a stored procedure called `update_order_status_with_notification` that:
- Accepts: order_id, new_status, updated_by
- Updates the order status
- Logs the change to an audit table (create the audit table if needed)
- If status changes to 'Completed', automatically updates customer statistics
- Returns a message indicating all actions taken

**Expected Output Format:**
- Returns a string with all actions performed


### Problem 8: Data Validation and Business Rules
Create a stored procedure called `apply_discount_to_order` that:
- Accepts: order_id, discount_percent
- Validates that the order exists
- Validates that discount_percent is between 0 and 50
- Validates that the order status is 'Pending' (only pending orders can have discounts applied)
- Calculates the new total amount with discount
- Updates the order total_amount
- Returns the original amount, discount amount, and new total

**Expected Output Format:**
- Returns a string with discount calculation details


## Solutions to Practice Problems

### Solution 1: Basic Stored Procedure


In [ ]:
-- Solution 1: Basic Stored Procedure
CREATE OR REPLACE PROCEDURE get_product_info(product_id_param INT)
RETURNS TABLE (
    product_name VARCHAR(100),
    category VARCHAR(50),
    price DECIMAL(10, 2),
    stock_quantity INT
)
LANGUAGE SQL
AS
$$
    SELECT 
        product_name,
        category,
        price,
        stock_quantity
    FROM products
    WHERE product_id = product_id_param;
$$;

-- Test the procedure
CALL get_product_info(1001);


### Solution 2: Stored Procedure with Variables


In [ ]:
-- Solution 2: Stored Procedure with Variables
CREATE OR REPLACE PROCEDURE calculate_order_total(order_id_param INT)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        original_amount DECIMAL(10, 2);
        discount_amount DECIMAL(10, 2);
        final_amount DECIMAL(10, 2);
        result_message STRING;
    BEGIN
        -- Get original amount
        SELECT total_amount INTO original_amount
        FROM orders
        WHERE order_id = order_id_param;
        
        IF original_amount IS NULL THEN
            RETURN 'Error: Order ' || order_id_param || ' does not exist.';
        END IF;
        
        -- Calculate discount if amount > $200
        IF original_amount > 200 THEN
            SET discount_amount = original_amount * 0.10;
            SET final_amount = original_amount - discount_amount;
            SET result_message = 'Original: $' || original_amount || 
                               ', Discount (10%): $' || discount_amount ||
                               ', Final: $' || final_amount;
        ELSE
            SET discount_amount = 0;
            SET final_amount = original_amount;
            SET result_message = 'Original: $' || original_amount || 
                               ', No discount applied, Final: $' || final_amount;
        END IF;
        
        RETURN result_message;
    END;
$$;

-- Test the procedure
CALL calculate_order_total(101);
CALL calculate_order_total(104);


### Solution 3: Control Flow (IF-ELSE)


In [ ]:
-- Solution 3: Control Flow (IF-ELSE)
CREATE OR REPLACE PROCEDURE assign_customer_tier(customer_id_param INT)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        total_spent DECIMAL(10, 2);
        new_tier VARCHAR(20);
        current_tier VARCHAR(20);
        result_message STRING;
    BEGIN
        -- Get current tier
        SELECT customer_tier INTO current_tier
        FROM customers
        WHERE customer_id = customer_id_param;
        
        IF current_tier IS NULL THEN
            RETURN 'Error: Customer ' || customer_id_param || ' does not exist.';
        END IF;
        
        -- Calculate total spent
        SELECT COALESCE(SUM(total_amount), 0) INTO total_spent
        FROM orders
        WHERE customer_id = customer_id_param AND status = 'Completed';
        
        -- Assign tier based on spending
        IF total_spent >= 500 THEN
            SET new_tier = 'Gold';
        ELSEIF total_spent >= 200 THEN
            SET new_tier = 'Silver';
        ELSE
            SET new_tier = 'Bronze';
        END IF;
        
        -- Update tier if changed
        IF current_tier != new_tier THEN
            UPDATE customers
            SET customer_tier = new_tier
            WHERE customer_id = customer_id_param;
            
            SET result_message = 'Customer tier updated from ' || current_tier ||
                               ' to ' || new_tier || 
                               ' (Total spent: $' || total_spent || ')';
        ELSE
            SET result_message = 'Customer tier remains ' || current_tier ||
                               ' (Total spent: $' || total_spent || ')';
        END IF;
        
        RETURN result_message;
    END;
$$;

-- Test the procedure
CALL assign_customer_tier(1);
CALL assign_customer_tier(2);
CALL assign_customer_tier(3);


### Solution 4: Error Handling


In [ ]:
-- Solution 4: Error Handling
CREATE OR REPLACE PROCEDURE safe_delete_order(order_id_param INT)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        order_exists INT;
        order_status_val VARCHAR(20);
        result_message STRING;
    BEGIN
        -- Check if order exists
        SELECT COUNT(*), MAX(status) INTO order_exists, order_status_val
        FROM orders
        WHERE order_id = order_id_param;
        
        IF order_exists = 0 THEN
            RETURN 'Error: Order ' || order_id_param || ' does not exist.';
        END IF;
        
        -- Check if order status is Pending
        IF order_status_val != 'Pending' THEN
            RETURN 'Error: Cannot delete order ' || order_id_param || 
                   '. Only pending orders can be deleted. Current status: ' || order_status_val;
        END IF;
        
        -- Delete the order
        BEGIN
            DELETE FROM orders
            WHERE order_id = order_id_param;
            
            SET result_message = 'Order ' || order_id_param || ' deleted successfully.';
            RETURN result_message;
        EXCEPTION
            WHEN OTHER THEN
                RETURN 'Error deleting order: ' || SQLERRM;
        END;
    END;
$$;

-- Test the procedure
-- First, check which orders are pending
SELECT order_id, status FROM orders WHERE status = 'Pending';

-- Try to delete a pending order
CALL safe_delete_order(103);

-- Try to delete a completed order (should fail)
CALL safe_delete_order(101);

-- Try to delete non-existent order (should fail)
CALL safe_delete_order(999);


### Solution 5: Complex Stored Procedure


In [ ]:
-- Solution 5: Complex Stored Procedure
CREATE OR REPLACE PROCEDURE process_customer_order(
    customer_id_param INT,
    order_date_param DATE,
    total_amount_param DECIMAL(10, 2),
    status_param VARCHAR(20)
)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        customer_exists INT;
        new_order_id INT;
        result_message STRING;
    BEGIN
        -- Validate customer exists
        SELECT COUNT(*) INTO customer_exists
        FROM customers
        WHERE customer_id = customer_id_param;
        
        IF customer_exists = 0 THEN
            RETURN 'Error: Customer ' || customer_id_param || ' does not exist.';
        END IF;
        
        -- Validate total_amount is positive
        IF total_amount_param IS NULL OR total_amount_param <= 0 THEN
            RETURN 'Error: Total amount must be positive.';
        END IF;
        
        -- Validate order_date is not in the future
        IF order_date_param > CURRENT_DATE() THEN
            RETURN 'Error: Order date cannot be in the future.';
        END IF;
        
        -- Generate new order_id
        SELECT COALESCE(MAX(order_id), 0) + 1 INTO new_order_id
        FROM orders;
        
        -- Insert the new order
        BEGIN
            INSERT INTO orders (order_id, customer_id, order_date, total_amount, status)
            VALUES (new_order_id, customer_id_param, order_date_param, total_amount_param, status_param);
            
            -- Update customer statistics (trigger-like behavior)
            CALL recalculate_customer_stats(customer_id_param);
            
            SET result_message = 'Order ' || new_order_id || 
                               ' created successfully for customer ' || customer_id_param;
            RETURN result_message;
        EXCEPTION
            WHEN OTHER THEN
                RETURN 'Error creating order: ' || SQLERRM;
        END;
    END;
$$;

-- Test the procedure
CALL process_customer_order(2, '2023-10-25', 150.00, 'Pending');

-- Test with invalid data
CALL process_customer_order(999, '2023-10-25', 150.00, 'Pending');
CALL process_customer_order(1, '2024-12-31', 150.00, 'Pending');
CALL process_customer_order(1, '2023-10-25', -50.00, 'Pending');


### Solution 6: Multiple Parameters and Complex Logic


In [ ]:
-- Solution 6: Multiple Parameters and Complex Logic
CREATE OR REPLACE PROCEDURE get_customer_orders_summary(
    customer_id_param INT,
    start_date DATE,
    end_date DATE
)
RETURNS TABLE (
    customer_id INT,
    customer_name VARCHAR,
    customer_tier VARCHAR(20),
    total_orders INT,
    total_amount_spent DECIMAL(10, 2),
    average_order_value DECIMAL(10, 2),
    completed_orders INT,
    pending_orders INT
)
LANGUAGE SQL
AS
$$
    DECLARE
        customer_name_val VARCHAR;
        customer_tier_val VARCHAR(20);
    BEGIN
        -- Get customer information
        SELECT 
            first_name || ' ' || last_name,
            customer_tier
        INTO customer_name_val, customer_tier_val
        FROM customers
        WHERE customer_id = customer_id_param;
        
        IF customer_name_val IS NULL THEN
            RETURN TABLE(
                SELECT 
                    NULL::INT AS customer_id,
                    NULL::VARCHAR AS customer_name,
                    NULL::VARCHAR(20) AS customer_tier,
                    NULL::INT AS total_orders,
                    NULL::DECIMAL(10, 2) AS total_amount_spent,
                    NULL::DECIMAL(10, 2) AS average_order_value,
                    NULL::INT AS completed_orders,
                    NULL::INT AS pending_orders
                WHERE FALSE
            );
        END IF;
        
        -- Return summary statistics
        RETURN TABLE(
            SELECT 
                customer_id_param AS customer_id,
                customer_name_val AS customer_name,
                customer_tier_val AS customer_tier,
                COUNT(*) AS total_orders,
                COALESCE(SUM(total_amount), 0) AS total_amount_spent,
                COALESCE(AVG(total_amount), 0) AS average_order_value,
                SUM(CASE WHEN status = 'Completed' THEN 1 ELSE 0 END) AS completed_orders,
                SUM(CASE WHEN status = 'Pending' THEN 1 ELSE 0 END) AS pending_orders
            FROM orders
            WHERE customer_id = customer_id_param
                AND order_date BETWEEN start_date AND end_date
        );
    END;
$$;

-- Test the procedure
CALL get_customer_orders_summary(1, '2023-01-01', '2023-12-31');
CALL get_customer_orders_summary(2, '2023-06-01', '2023-09-30');


### Solution 7: Trigger-like Functionality


In [ ]:
-- Solution 7: Trigger-like Functionality
CREATE OR REPLACE PROCEDURE update_order_status_with_notification(
    order_id_param INT,
    new_status_param VARCHAR(20),
    updated_by_param VARCHAR(100)
)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        order_exists INT;
        old_status_val VARCHAR(20);
        customer_id_val INT;
        actions_taken STRING DEFAULT '';
    BEGIN
        -- Check if order exists
        SELECT COUNT(*), MAX(status), MAX(customer_id) 
        INTO order_exists, old_status_val, customer_id_val
        FROM orders
        WHERE order_id = order_id_param;
        
        IF order_exists = 0 THEN
            RETURN 'Error: Order ' || order_id_param || ' does not exist.';
        END IF;
        
        -- Update order status
        UPDATE orders
        SET status = new_status_param
        WHERE order_id = order_id_param;
        
        SET actions_taken = 'Order status updated from ' || old_status_val || 
                           ' to ' || new_status_param || '. ';
        
        -- Log the change (audit logging)
        CALL log_order_change(
            order_id_param,
            'UPDATE',
            old_status_val,
            new_status_param,
            updated_by_param
        );
        
        SET actions_taken = actions_taken || 'Change logged to audit table. ';
        
        -- If status changed to Completed, update customer statistics (trigger-like)
        IF new_status_param = 'Completed' AND old_status_val != 'Completed' THEN
            CALL recalculate_customer_stats(customer_id_val);
            SET actions_taken = actions_taken || 'Customer statistics updated. ';
        END IF;
        
        RETURN actions_taken;
    EXCEPTION
        WHEN OTHER THEN
            RETURN 'Error updating order: ' || SQLERRM;
    END;
$$;

-- Test the procedure
CALL update_order_status_with_notification(106, 'Completed', 'admin@company.com');

-- View audit log
SELECT * FROM order_audit_log ORDER BY change_timestamp DESC LIMIT 5;


### Solution 8: Data Validation and Business Rules


In [ ]:
-- Solution 8: Data Validation and Business Rules
CREATE OR REPLACE PROCEDURE apply_discount_to_order(
    order_id_param INT,
    discount_percent DECIMAL(5, 2)
)
RETURNS STRING
LANGUAGE SQL
AS
$$
    DECLARE
        order_exists INT;
        order_status_val VARCHAR(20);
        original_amount DECIMAL(10, 2);
        discount_amount DECIMAL(10, 2);
        new_total DECIMAL(10, 2);
        validation_errors STRING DEFAULT '';
        result_message STRING;
    BEGIN
        -- Validate discount_percent
        IF discount_percent IS NULL OR discount_percent < 0 OR discount_percent > 50 THEN
            SET validation_errors = validation_errors || 
                'Discount percent must be between 0 and 50. ';
        END IF;
        
        -- Check if order exists
        SELECT COUNT(*), MAX(status), MAX(total_amount)
        INTO order_exists, order_status_val, original_amount
        FROM orders
        WHERE order_id = order_id_param;
        
        IF order_exists = 0 THEN
            SET validation_errors = validation_errors || 
                'Order ' || order_id_param || ' does not exist. ';
        ELSE
            -- Validate order status is Pending
            IF order_status_val != 'Pending' THEN
                SET validation_errors = validation_errors || 
                    'Only pending orders can have discounts applied. Current status: ' || 
                    order_status_val || '. ';
            END IF;
        END IF;
        
        -- If validation errors exist, return them
        IF validation_errors != '' THEN
            RETURN 'Validation Error: ' || validation_errors;
        END IF;
        
        -- Calculate discount and new total
        SET discount_amount = original_amount * (discount_percent / 100);
        SET new_total = original_amount - discount_amount;
        
        -- Update order
        BEGIN
            UPDATE orders
            SET total_amount = new_total
            WHERE order_id = order_id_param;
            
            SET result_message = 'Discount applied. Original: $' || original_amount ||
                               ', Discount (' || discount_percent || '%): $' || discount_amount ||
                               ', New Total: $' || new_total;
            RETURN result_message;
        EXCEPTION
            WHEN OTHER THEN
                RETURN 'Error applying discount: ' || SQLERRM;
        END;
    END;
$$;

-- Test the procedure
-- First, create a pending order
CALL insert_validated_order(114, 1, '2023-10-26', 250.00, 'Pending');

-- Apply discount to pending order
CALL apply_discount_to_order(114, 15.00);

-- Try to apply discount to completed order (should fail)
CALL apply_discount_to_order(101, 10.00);

-- Try with invalid discount percent (should fail)
CALL apply_discount_to_order(114, 75.00);
